# บทที่ 3: การวิเคราะห์และสร้างตัวแปร (Analysis & Feature Engineering)

---

**บริษัท:** บริษัท แลนด์ แอนด์ เฮ้าส์ จำกัด (มหาชน) | Land & Houses  
**ผู้บรรยาย:** ผู้ช่วยศาสตราจารย์ ดร.วสิศ ลิ้มประเสริฐ  
**วันที่:** 11 พฤศจิกายน 2025

## Executive Summary

ในบทนี้ เราจะเรียนรู้การแปลงข้อมูลดิบให้กลายเป็นข้อมูลที่มีประโยชน์และเข้าใจง่ายขึ้น ผ่านกระบวนการ **Feature Engineering** ซึ่งเป็นหัวใจสำคัญของงาน Data Science หลังจากที่เรามี Master Table ที่สะอาดและรวมข้อมูลครบถ้วนแล้ว ข้อมูลดิบอย่าง FDPDUE01 (จำนวนวันค้างชำระ) ที่เป็นตัวเลขเพียงอย่างเดียวอาจวิเคราะห์ได้ยาก เราจะเรียนรู้วิธีสร้างตัวแปรใหม่ เช่น DPD Bucket, Stage Name, และ Loan Age เพื่อให้ข้อมูลมีความหมายมากขึ้น นอกจากนี้ เราจะเรียนรู้เทคนิคการวิเคราะห์และสรุปผลด้วย GroupBy และ Pivot Table เพื่อสร้างรายงานที่ผู้บริหารสามารถนำไปใช้ตัดสินใจได้ทันที บทเรียนนี้จะช่วยให้คุณสามารถแปลงข้อมูลดิบให้กลายเป็น Insights ที่มีคุณค่าได้อย่างมืออาชีพ

## Key Takeaways

- **Feature Engineering คือหัวใจของ Data Science:** การสร้างตัวแปรใหม่จากข้อมูลเดิมช่วยให้วิเคราะห์และทำความเข้าใจข้อมูลได้ง่ายขึ้น
- **เทคนิค Mapping, Binning และ Conditional:** เครื่องมือสำคัญในการแปลงข้อมูลให้อยู่ในรูปแบบที่เหมาะสมกับการวิเคราะห์
- **การทำงานกับวันที่:** สามารถสร้างตัวแปรที่มีประโยชน์เช่น อายุสินเชื่อ, เวลาที่เหลือ, และข้อมูลตามช่วงเวลา
- **GroupBy และ Pivot Table:** เครื่องมือที่ทรงพลังสำหรับการสรุปและวิเคราะห์ข้อมูลแบบกลุ่ม
- **การส่งออกรายงาน:** สามารถสร้างรายงาน Excel ที่สวยงามและพร้อมนำเสนอได้ทันที

## Dependencies & Setup

ติดตั้ง libraries ที่จำเป็นสำหรับ Workshop นี้

In [723]:
# ติดตั้ง dependencies (รันครั้งเดียว)
# !pip install pandas numpy openpyxl --quiet

In [724]:
# Import libraries ที่จำเป็น
import pandas as pd
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Pandas version: 2.3.0
NumPy version: 1.24.3


---

# Main Story

 **"ตอนนี้เรามี 'Master Table' (df_merged) ที่สะอาดและรวมข้อมูลทุกอย่างแล้ว แต่ข้อมูลดิบๆ เช่น FDPDUE01 (27 วัน, 98 วัน) วิเคราะห์ได้ยาก ในบทนี้ เราจะ 'สร้างตัวแปรใหม่' (Feature Engineering) เช่น 'DPD Bucket', 'Stage Name' และ 'Loan Age' เพื่อทำให้ข้อมูล 'มีประโยชน์' และง่ายต่อการสรุปผล"**

---

## อ่านข้อมูล (Data)

In [725]:
# # สร้างข้อมูลตัวอย่างสำหรับ Workshop
# np.random.seed(42)

# n_records = 1000

# # สร้าง DataFrame ตัวอย่าง
# df_merged = pd.DataFrame({
#     'FACCNO': [f'ACC{i:05d}' for i in range(1, n_records + 1)],
#     'STAGE_CIF': np.random.choice([1, 2, 3], size=n_records, p=[0.7, 0.2, 0.1]),
#     'FDPDUE00': np.random.choice([0, 15, 35, 65, 95, 120], size=n_records, p=[0.6, 0.15, 0.1, 0.08, 0.05, 0.02]),
#     'FPRINCAM': np.random.uniform(100000, 10000000, size=n_records),
#     'FPRODTY': np.random.choice(['HL', 'PL', 'AL', 'CC'], size=n_records, p=[0.4, 0.3, 0.2, 0.1]),
#     'FFLGBWFW': np.random.uniform(200000, 15000000, size=n_records),
#     'FORDATE': pd.date_range(start='2020-01-01', periods=n_records, freq='D'),
#     'FRPDATE': pd.Timestamp('2025-01-15'),
#     'FMATDATE': pd.date_range(start='2030-01-01', periods=n_records, freq='D')
# })

# print(f"สร้างข้อมูลตัวอย่าง {len(df_merged):,} รายการสำเร็จ")
# print(f"\nตัวอย่างข้อมูล 5 แถวแรก:")
# df_merged.head()

In [726]:
df_tran = pd.read_csv('Transection_20240731.csv', encoding='utf-8-sig', sep='|')
df_perf = pd.read_excel('Performance.xlsx')
df_merged = pd.merge(df_tran, df_perf, left_on='FCUSNO', right_on='CIF', how='inner')
df_merged.head()

,FRPDATE_x,FRLDATE,FORDATE,FMATDATE,FCUSNO,FACCNO,FLNTYP,FACCSTS,FPRODTY,FNPLFDTE,...,FSGMCOD,FCARID00,FDPDUE00,FCOMMFG,FRATING_ORI,FRATING_DTE,FRATING_RP,FRPDATE_y,CIF,STAGE_CIF
0,20240731,20070323,20070323,20460323,52005,5579906439,L,1,H1,0,...,HL,A,0,Y,5,NaN,5,20240731,52005,3
1,20240731,20070323,20070323,20460323,52005,5579906439,L,1,H1,0,...,HL,A,0,Y,5,NaN,5,20240831,52005,3
2,20240731,20070323,20070323,20460323,52005,5579906439,L,1,H1,0,...,HL,A,0,Y,5,NaN,5,20240930,52005,3
3,20240731,20070323,20070323,20460323,52005,5579906439,L,1,H1,0,...,HL,A,0,Y,5,NaN,5,20241031,52005,3
4,20240731,20070323,20070323,20460323,52005,5579906439,L,1,H1,0,...,HL,A,0,Y,5,NaN,5,20241130,52005,3


# ลบ columns  ซ้ำ

In [727]:
if 'FRPDATE_x' in df_merged.columns:
    df_merged.rename(columns={'FRPDATE_x': 'FRPDATE'}, inplace=True)
if 'FRPDATE_y' in df_merged.columns:
    df_merged.drop(columns=['FRPDATE_y'], inplace=True)

---

# 3.1 การสร้างตัวแปรใหม่ (Feature Engineering)

Feature Engineering คือกระบวนการสร้างตัวแปร (Features) ใหม่จากข้อมูลที่มีอยู่ เพื่อให้ข้อมูลมีความหมายมากขึ้นและง่ายต่อการวิเคราะห์

## 3.1.1 การจับคู่และแปลงข้อมูล (Mapping)

**เป้าหมาย:** แปลง STAGE_CIF ที่เป็นตัวเลข (1, 2, 3) ให้เป็นข้อความที่อ่านง่าย

**เครื่องมือ:** `.map()` - ใช้สำหรับแปลงค่าตาม dictionary ที่กำหนด

In [728]:
# สร้าง mapping dictionary
example_map = {1: 'A', 2: 'B', 3: 'C'}
example_series = pd.Series([1, 2, 3, 1, 2])

print("ข้อมูลเดิม:", example_series.tolist())
print("หลังใช้ .map():", example_series.map(example_map).tolist())

ข้อมูลเดิม: [1, 2, 3, 1, 2]
หลังใช้ .map(): ['A', 'B', 'C', 'A', 'B']


### Workshop 3.1: Mapping STAGE_CIF

In [729]:
# Workshop 3.1: แปลง STAGE_CIF เป็นชื่อ Stage ที่อ่านง่าย

# สร้าง mapping dictionary
stage_map = {
    1: '1. Performing',
    2: '2. Under-performing',
    3: '3. NPL'
}

# แปลงข้อมูล
df_merged['Stage_Name'] = df_merged['STAGE_CIF'].map(stage_map)

# จัดการค่าว่าง (ถ้ามี)
df_merged['Stage_Name'].fillna('0. Unknown', inplace=True)

In [730]:
print("สร้างคอลัมน์ Stage_Name สำเร็จ\n")
df_merged[['STAGE_CIF', 'Stage_Name']].sample(10)

สร้างคอลัมน์ Stage_Name สำเร็จ



,STAGE_CIF,Stage_Name
22,1,1. Performing
208,1,1. Performing
61,1,1. Performing
60,1,1. Performing
70,1,1. Performing
174,1,1. Performing
177,1,1. Performing
126,1,1. Performing
229,2,2. Under-performing
245,1,1. Performing


In [731]:
print(f"\nสรุปจำนวนตาม Stage:")
df_merged['Stage_Name'].value_counts().sort_index()


สรุปจำนวนตาม Stage:


Stage_Name
1. Performing          240
2. Under-performing     14
3. NPL                  12
Name: count, dtype: int64

## 3.1.2 การสร้างตัวแปรจากเงื่อนไข (Conditional)

**เป้าหมาย:** สร้าง Flag เพื่อระบุว่าบัญชีนั้นค้างชำระหรือไม่

**เครื่องมือ:** `np.where()` - เทียบเท่ากับฟังก์ชัน IF ใน Excel

In [732]:
# สร้างข้อมูลตัวอย่าง
scores = pd.Series([85, 92, 78, 65, 88])
result = np.where(scores >= 80, 'Pass', 'Fail')

print(f"คะแนน: {scores.tolist()}")
print(f"ผลลัพธ์: {result.tolist()}")

คะแนน: [85, 92, 78, 65, 88]
ผลลัพธ์: ['Pass', 'Pass', 'Fail', 'Fail', 'Pass']


### Workshop 3.2: สร้าง Is_Overdue Flag

In [733]:
# สร้างเงื่อนไข: ถ้า FDPDUE01 > 0 แสดงว่าค้างชำระ
df_merged['Is_Overdue'] = np.where(df_merged['FDPDUE00'] > 0, True, False)

In [734]:
print("สร้างคอลัมน์ Is_Overdue สำเร็จ\n")
df_merged[['FDPDUE00', 'Is_Overdue']]

สร้างคอลัมน์ Is_Overdue สำเร็จ



,FDPDUE00,Is_Overdue
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
261,0,False
262,0,False
263,0,False
264,0,False


In [735]:
a = df_merged['Is_Overdue'].value_counts()
a

Is_Overdue
False    254
True      12
Name: count, dtype: int64

In [736]:
print(f"\nสัดส่วนค้างชำระ: {df_merged['Is_Overdue'].mean()*100:.2f}%")


สัดส่วนค้างชำระ: 4.51%


## 3.1.3 การจัดกลุ่มตัวเลข (Binning)

**เป้าหมาย:** จัดกลุ่ม DPD (Days Past Due) เป็นช่วง (Buckets) เพื่อให้วิเคราะห์ง่ายขึ้น

**เครื่องมือ:** `pd.cut()` - เทียบเท่ากับ IF ซ้อนกันหลายชั้น หรือ VLOOKUP แบบ True ใน Excel

In [737]:
# จัดกลุ่มอายุ
ages = pd.Series([5, 15, 25, 35, 45, 55])
age_bins = [0, 18, 35, 60, 100]
age_labels = ['เด็ก', 'วัยรุ่น', 'วัยทำงาน', 'วัยเกษียณ']

age_groups = pd.cut(ages, bins=age_bins, labels=age_labels)

In [738]:
print(f"อายุ: {ages.tolist()}")
print(f"กลุ่ม: {age_groups.tolist()}")

อายุ: [5, 15, 25, 35, 45, 55]
กลุ่ม: ['เด็ก', 'เด็ก', 'วัยรุ่น', 'วัยรุ่น', 'วัยทำงาน', 'วัยทำงาน']


### Workshop 3.3: สร้าง DPD_Bucket

In [739]:
# Workshop 3.3: จัดกลุ่ม FDPDUE01 เป็น DPD Buckets

# กำหนดช่วงและป้ายกำกับ
bins = [-1, 0, 30, 60, 90, float('inf')]
labels = [
    '0. No DPD',
    '1. 1-30 Days',
    '2. 31-60 Days',
    '3. 61-90 Days',
    '4. 90+ Days'
]

# สร้าง DPD_Bucket
df_merged['DPD_Bucket'] = pd.cut(
    df_merged['FDPDUE00'], 
    bins=bins, 
    labels=labels, 
    right=True
)

In [740]:
print("สร้างคอลัมน์ DPD_Bucket สำเร็จ\n")
df_merged[['FDPDUE00', 'DPD_Bucket']].head()

สร้างคอลัมน์ DPD_Bucket สำเร็จ



,FDPDUE00,DPD_Bucket
0,0,0. No DPD
1,0,0. No DPD
2,0,0. No DPD
3,0,0. No DPD
4,0,0. No DPD


In [741]:
print(f"\nสรุปจำนวนตาม DPD Bucket:")
df_merged['DPD_Bucket'].value_counts().sort_index()


สรุปจำนวนตาม DPD Bucket:


DPD_Bucket
0. No DPD        254
1. 1-30 Days       0
2. 31-60 Days      0
3. 61-90 Days      0
4. 90+ Days       12
Name: count, dtype: int64

## 3.1.4 การใช้ .apply() กับฟังก์ชัน

**เป้าหมาย:** ใช้ฟังก์ชันที่เราสร้างเองกับข้อมูลทั้งคอลัมน์

**เครื่องมือ:** `.apply()` - สามารถใช้ได้ทั้งกับฟังก์ชันปกติและ lambda function

In [742]:
numbers = pd.Series([1, 2, 3, 4, 5])
squared = numbers.apply(lambda x: x ** 2)

print(f"ตัวเลขเดิม: {numbers.tolist()}")
print(f"ยกกำลังสอง: {squared.tolist()}")

ตัวเลขเดิม: [1, 2, 3, 4, 5]
ยกกำลังสอง: [1, 4, 9, 16, 25]


### Workshop 3.4: ใช้ .apply() แบบต่างๆ

In [743]:
# 1. สร้างฟังก์ชันจำแนก Stage จาก DPD
def classify_stage(dpd_days):
    """จำแนก Stage ตามจำนวนวันค้างชำระ"""
    if dpd_days > 90:
        return "3. NPL"
    elif dpd_days > 30:
        return "2. Under-performing"
    else:
        return "1. Performing"

# ใช้ฟังก์ชันกับข้อมูล
df_merged['Stage_from_DPD'] = df_merged['FDPDUE00'].apply(classify_stage)

In [744]:
df_merged[['FDPDUE00', 'Stage_from_DPD']].head(10)

,FDPDUE00,Stage_from_DPD
0,0,1. Performing
1,0,1. Performing
2,0,1. Performing
3,0,1. Performing
4,0,1. Performing
5,0,1. Performing
6,0,1. Performing
7,0,1. Performing
8,0,1. Performing
9,0,1. Performing


In [745]:
# 2. ตัวอย่างการใช้ lambda: สร้าง Flag สินเชื่อวงเงินสูง
df_merged['High_Value_Flag'] = df_merged['FPRINCAM'].apply(
    lambda x: 'High Value' if x > 5000000 else 'Standard'
)

In [746]:
df_merged['High_Value_Flag'].value_counts()

High_Value_Flag
Standard      229
High Value     37
Name: count, dtype: int64

In [747]:
df_merged['FFLGBWFW'] = pd.to_numeric(df_merged['FFLGBWFW'], errors='coerce')
df_merged['FPRINCAM'] = pd.to_numeric(df_merged['FPRINCAM'], errors='coerce')

In [748]:
# 3. ตัวอย่างขั้นสูง: .apply(axis=1) ใช้ข้อมูลหลายคอลัมน์
def calculate_ratio(row):
    """คำนวณอัตราส่วนหนี้ต่อวงเงิน"""
    if row['FFLGBWFW'] > 0:
        return row['FPRINCAM'] / row['FFLGBWFW']
    else:
        return np.nan

df_merged['Debt_to_Limit_Ratio'] = df_merged.apply(calculate_ratio, axis=1)

In [749]:
df_merged[['FPRINCAM', 'FFLGBWFW', 'Debt_to_Limit_Ratio']].head(10)

,FPRINCAM,FFLGBWFW,Debt_to_Limit_Ratio
0,818366.72,NaN,NaN
1,818366.72,NaN,NaN
2,818366.72,NaN,NaN
3,818366.72,NaN,NaN
4,818366.72,NaN,NaN
5,818366.72,NaN,NaN
6,818366.72,NaN,NaN
7,818366.72,NaN,NaN
8,818366.72,NaN,NaN
9,818366.72,NaN,NaN


In [750]:
print(f"\nค่าเฉลี่ย Debt to Limit Ratio: {df_merged['Debt_to_Limit_Ratio'].mean():.2%}")


ค่าเฉลี่ย Debt to Limit Ratio: nan%


## 3.1.5 การสร้างตัวแปรจากวันที่ (Date Feature Engineering)

**เป้าหมาย:** สร้างตัวแปรที่มีประโยชน์จากข้อมูลวันที่

**เครื่องมือ:** `.dt` accessor - ใช้เข้าถึงคุณสมบัติของ datetime

In [751]:
dates = pd.Series(pd.date_range('2024-01-01', periods=5))
print(f"วันที่: {dates.dt.date.tolist()}")
print(f"ปี: {dates.dt.year.tolist()}")
print(f"เดือน: {dates.dt.month.tolist()}")
print(f"ไตรมาส: {dates.dt.quarter.tolist()}")

วันที่: [datetime.date(2024, 1, 1), datetime.date(2024, 1, 2), datetime.date(2024, 1, 3), datetime.date(2024, 1, 4), datetime.date(2024, 1, 5)]
ปี: [2024, 2024, 2024, 2024, 2024]
เดือน: [1, 1, 1, 1, 1]
ไตรมาส: [1, 1, 1, 1, 1]


### Workshop 3.5: สร้างตัวแปรจากวันที่

In [752]:
for col in ['FRPDATE', 'FORDATE', 'FMATDATE']:
    df_merged[col] = pd.to_datetime(df_merged[col].astype(str), format='%Y%m%d', errors='coerce')

In [753]:
# สร้างคอลัมน์ Loan_Age และ Remaining_Tenor
df_merged['Loan_Age_Days'] = (df_merged['FRPDATE'] - df_merged['FORDATE']).dt.days
df_merged['Remaining_Tenor_Days'] = (df_merged['FMATDATE'] - df_merged['FRPDATE']).dt.days
df_merged['Loan_Orig_Year'] = df_merged['FORDATE'].dt.year
df_merged['Loan_Orig_Quarter'] = df_merged['FORDATE'].dt.quarter

In [754]:
df_merged.rename(columns={
    'FRPDATE_x': 'FRPDATE',
    'FRPDATE_y': 'FRPDATE_perf'
}, inplace=True)


In [755]:
df_merged[[
    'FRPDATE', 'FORDATE', 'FMATDATE',
    'Loan_Age_Days', 'Remaining_Tenor_Days',
    'Loan_Orig_Year', 'Loan_Orig_Quarter']].head()

,FRPDATE,FORDATE,FMATDATE,Loan_Age_Days,Remaining_Tenor_Days,Loan_Orig_Year,Loan_Orig_Quarter
0,2024-07-31,2007-03-23,2046-03-23,6340,7905,2007,1
1,2024-07-31,2007-03-23,2046-03-23,6340,7905,2007,1
2,2024-07-31,2007-03-23,2046-03-23,6340,7905,2007,1
3,2024-07-31,2007-03-23,2046-03-23,6340,7905,2007,1
4,2024-07-31,2007-03-23,2046-03-23,6340,7905,2007,1


In [756]:
print(f"\nสถิติอายุสินเชื่อ:")
print(f"เฉลี่ย: {df_merged['Loan_Age_Days'].mean():.0f} วัน")
print(f"ต่ำสุด: {df_merged['Loan_Age_Days'].min():.0f} วัน")
print(f"สูงสุด: {df_merged['Loan_Age_Days'].max():.0f} วัน")


สถิติอายุสินเชื่อ:
เฉลี่ย: 1293 วัน
ต่ำสุด: 1 วัน
สูงสุด: 6340 วัน


---

# 3.2 การวิเคราะห์และสรุปผล (Aggregation & Analysis)

หลังจากสร้างตัวแปรใหม่แล้ว เราจะมาเรียนรู้การสรุปและวิเคราะห์ข้อมูลเพื่อสร้างรายงาน

## 3.2.1 การจัดกลุ่มข้อมูล (Grouping)

**เป้าหมาย:** สรุปยอดหนี้รวม (SUM) และยอดหนี้เฉลี่ย (MEAN) โดยแบ่งตาม Stage และ DPD Bucket

**เครื่องมือ:** `.groupby()` - เทียบเท่า Pivot Table ใน Excel

In [757]:
# สร้างข้อมูลตัวอย่าง
demo_df = pd.DataFrame({
    'Category': ['A', 'B', 'A', 'B', 'A'],
    'Value': [10, 20, 30, 40, 50]
})

In [758]:
print("ข้อมูลเดิม:")
demo_df

ข้อมูลเดิม:


,Category,Value
0,A,10
1,B,20
2,A,30
3,B,40
4,A,50


In [759]:
print("\nสรุปผลตาม Category:")
demo_df.groupby('Category')['Value'].sum()


สรุปผลตาม Category:


Category
A    90
B    60
Name: Value, dtype: int64

### Workshop 3.6: GroupBy พื้นฐาน

In [760]:
# 1. สรุปยอดหนี้รวมตาม Stage
print("1. สรุปยอดหนี้ตาม Stage_Name:\n")
report_by_stage = df_merged.groupby('Stage_Name')['FPRINCAM'].agg(['sum', 'mean', 'count'])
report_by_stage.columns = ['ยอดหนี้รวม', 'ยอดหนี้เฉลี่ย', 'จำนวนบัญชี']
report_by_stage['ยอดหนี้รวม'] = report_by_stage['ยอดหนี้รวม'].apply(lambda x: f'{x:,.0f}')
report_by_stage['ยอดหนี้เฉลี่ย'] = report_by_stage['ยอดหนี้เฉลี่ย'].apply(lambda x: f'{x:,.0f}')
report_by_stage

1. สรุปยอดหนี้ตาม Stage_Name:



,ยอดหนี้รวม,ยอดหนี้เฉลี่ย,จำนวนบัญชี
Stage_Name,,,
1. Performing,"486,051,876","2,025,216",240
2. Under-performing,"264,198,609","18,871,329",14
3. NPL,"9,820,401","818,367",12


In [761]:
# 2. สรุปยอดหนี้รวมตาม DPD Bucket
print("\n2. สรุปยอดหนี้ตาม DPD_Bucket:\n")
report_by_dpd_bucket = df_merged.groupby('DPD_Bucket')['FPRINCAM'].agg(['sum', 'count'])
report_by_dpd_bucket.columns = ['ยอดหนี้รวม', 'จำนวนบัญชี']
report_by_dpd_bucket['ยอดหนี้รวม'] = report_by_dpd_bucket['ยอดหนี้รวม'].apply(lambda x: f'{x:,.0f}')
report_by_dpd_bucket


2. สรุปยอดหนี้ตาม DPD_Bucket:



,ยอดหนี้รวม,จำนวนบัญชี
DPD_Bucket,,
0. No DPD,"583,190,886",254
1. 1-30 Days,0,0
2. 31-60 Days,0,0
3. 61-90 Days,0,0
4. 90+ Days,"176,880,000",12


## 3.2.2 การ Grouping ขั้นสูง (Advanced Grouping)

**เป้าหมาย:** Group by หลายคอลัมน์ และใช้ฟังก์ชันสรุปผลที่แตกต่างกันในแต่ละคอลัมน์

**เครื่องมือ:** `.agg()` ด้วย Dictionary

### Workshop 3.7: Advanced GroupBy

In [762]:
# 1. Group by 2 ระดับ (Stage และ Product Type)
print("1. สรุปยอดหนี้ตาม Stage และ Product Type:\n")
report_multi_level = df_merged.groupby(['Stage_Name', 'FPRODTY'])['FPRINCAM'].agg(['sum', 'count'])
report_multi_level.columns = ['ยอดหนี้รวม', 'จำนวนบัญชี']
report_multi_level.head()

1. สรุปยอดหนี้ตาม Stage และ Product Type:



ยอดหนี้รวม  จำนวนบัญชี
Stage_Name    FPRODTY                          
1. Performing F2       1.768800e+08          12
              H1       2.994250e+08          96
              P4       4.060015e+05          12
              PA       3.478286e+06          12
              PE       1.700110e+06          26

In [763]:

print("\n2. สรุปข้อมูลแบบครบถ้วน:\n")
summary_agg = df_merged.groupby('Stage_Name').agg(
    Total_Principal=('FPRINCAM', 'sum'),
    Avg_DPD=('FDPDUE00', 'mean'),
    Avg_Loan_Age=('Loan_Age_Days', 'mean'),
    Num_Accounts=('FACCNO', 'count')
)
summary_agg


2. สรุปข้อมูลแบบครบถ้วน:



,Total_Principal,Avg_DPD,Avg_Loan_Age,Num_Accounts
Stage_Name,,,,
1. Performing,4.860519e+08,59.4,1101.408333,240
2. Under-performing,2.641986e+08,0.0,255.642857,14
3. NPL,9.820401e+06,0.0,6340.000000,12


In [764]:
summary_display = summary_agg.copy()
summary_display['Total_Principal'] = summary_display['Total_Principal'].apply(lambda x: f'{x:,.0f}')
summary_display['Avg_DPD'] = summary_display['Avg_DPD'].apply(lambda x: f'{x:.1f}')
summary_display['Avg_Loan_Age'] = summary_display['Avg_Loan_Age'].apply(lambda x: f'{x:.0f}')

summary_display

,Total_Principal,Avg_DPD,Avg_Loan_Age,Num_Accounts
Stage_Name,,,,
1. Performing,"486,051,876",59.4,1101,240
2. Under-performing,"264,198,609",0.0,256,14
3. NPL,"9,820,401",0.0,6340,12


## 3.2.3 การสรุปผลด้วย Pivot Table

**เป้าหมาย:** สร้างตารางสรุปแบบไขว้ที่ผู้บริหารต้องการดู

**เครื่องมือ:** `pd.pivot_table()` - สร้างตารางสรุปแบบ 2 มิติ

### Workshop 3.8: สร้าง Pivot Table

In [ ]:
# สร้างตารางสรุป "ยอดหนี้รวม"
# แถว (index) = DPD_Bucket
# คอลัมน์ (columns) = FPRODTY (ประเภท Product)

pivot_report = df_merged.pivot_table(
    index='DPD_Bucket',
    columns='FPRODTY',
    values='FPRINCAM',
    aggfunc='sum',
    fill_value=0,
    margins=True 
)

In [766]:
print("ตารางสรุปยอดหนี้ตาม DPD Bucket และ Product Type:\n")

pivot_display = pivot_report.applymap(lambda x: f'{x:,.0f}' if pd.notnull(x) else '0')
pivot_display

ตารางสรุปยอดหนี้ตาม DPD Bucket และ Product Type:



FPRODTY,F2,F8,H1,P4,PA,PE,PG,PP,All
DPD_Bucket,,,,,,,,,
0. No DPD,0,"260,000,000","313,444,048","406,002","3,478,286","1,700,110","3,943,387","219,052","583,190,886"
1. 1-30 Days,0,0,0,0,0,0,0,0,0
2. 31-60 Days,0,0,0,0,0,0,0,0,0
3. 61-90 Days,0,0,0,0,0,0,0,0,0
4. 90+ Days,"176,880,000",0,0,0,0,0,0,0,"176,880,000"
All,"176,880,000","260,000,000","313,444,048","406,002","3,478,286","1,700,110","3,943,387","219,052","760,070,886"


## 3.2.4 การเรียงลำดับและส่งออก (Sorting & Export)

**เป้าหมาย:** เรียงลำดับข้อมูลและส่งออกเป็นไฟล์ Excel

**เครื่องมือ:** `sort_values()` และ `to_excel()`

### Workshop 3.9: Sorting & Export

In [767]:
# 1. เรียงลำดับตาราง summary_agg (จาก Workshop 3.7)
summary_agg_sorted = summary_agg.sort_values(by='Total_Principal', ascending=False)

In [768]:
print("1. ตารางสรุปเรียงตามยอดหนี้รวม (มาก -> น้อย):\n")
summary_display_sorted = summary_agg_sorted.copy()
summary_display_sorted['Total_Principal'] = summary_display_sorted['Total_Principal'].apply(lambda x: f'{x:,.0f}')
summary_display_sorted['Avg_DPD'] = summary_display_sorted['Avg_DPD'].apply(lambda x: f'{x:.1f}')
summary_display_sorted['Avg_Loan_Age'] = summary_display_sorted['Avg_Loan_Age'].apply(lambda x: f'{x:.0f}')
print(summary_display_sorted)

1. ตารางสรุปเรียงตามยอดหนี้รวม (มาก -> น้อย):

                    Total_Principal Avg_DPD Avg_Loan_Age  Num_Accounts
Stage_Name                                                            
1. Performing           486,051,876    59.4         1101           240
2. Under-performing     264,198,609     0.0          256            14
3. NPL                    9,820,401     0.0         6340            12


In [769]:
# 2. ส่งออกรายงานเดี่ยว
try:
    pivot_report.to_excel("Monthly_NPL_Report.xlsx", sheet_name="Pivot by DPD")
    print("\nส่งออกไฟล์ 'Monthly_NPL_Report.xlsx' สำเร็จ")
except Exception as e:
    print(f"\nไม่สามารถส่งออกไฟล์ได้: {e}")



ส่งออกไฟล์ 'Monthly_NPL_Report.xlsx' สำเร็จ


In [770]:
# 3. ส่งออกหลายตารางในไฟล์เดียว
try:
    with pd.ExcelWriter('Full_NPL_Report.xlsx') as writer:
        pivot_report.to_excel(writer, sheet_name='Pivot_by_DPD')
        summary_agg_sorted.to_excel(writer, sheet_name='Summary_by_Stage')
    print("\nส่งออกไฟล์ 'Full_NPL_Report.xlsx' สำเร็จ")
except Exception as e:
    print(f"\nไม่สามารถส่งออกไฟล์ได้: {e}")


ส่งออกไฟล์ 'Full_NPL_Report.xlsx' สำเร็จ


---

## 3.3 Mini-Game: ทบทวนความรู้ (Review Game)

**ภารกิจ:** จับคู่ "สิ่งที่เราอยากทำ" กับ "โค้ด Pandas ที่ถูกต้อง"

### สิ่งที่เราอยากทำ (Tasks):

| ข้อ | Task | **คำตอบของคุณ** |
|-----|------|-----------------|
| 1 | สร้างตารางสรุปผลแบบไขว้ (แถว=DPD, คอลัมน์=Product) เหมือนใน Excel | ___ |
| 2 | แปลงตัวเลข `STAGE_CIF` (1, 2, 3) เป็นข้อความ ('Performing', 'NPL') | ___ |
| 3 | สร้าง Flag `Is_Overdue` (True/False) โดยใช้เงื่อนไข `FDPDUE01 > 0` | ___ |
| 4 | สรุปยอด `FPRINCAM` **รวม** โดยจัดกลุ่มตาม `Stage_Name` | ___ |
| 5 | ซอย `FDPDUE01` (ตัวเลข) ออกเป็น **"ช่วง"** (เช่น '1-30 Days', '31-60 Days') | ___ |
| 6 | คำนวณ **"อายุสินเชื่อ"** (เป็นวัน) จาก `FRPDATE` และ `FORDATE` | ___ |
| 7 | เรียงลำดับตาราง `summary_agg` ตามคอลัมน์ `Total_Principal` **จากมากไปน้อย** | ___ |
| 8 | สรุปผล**หลายแบบ** (Sum ของ `FPRINCAM` + **Mean** ของ `FDPDUE01`) โดยจัดกลุ่มตาม `Stage_Name` | ___ |

### โค้ด Pandas (Code Snippets):

| ตัวเลือก | โค้ด |
|---------|------|
| **A** | `pd.cut(df_merged['FDPDUE01'], bins=bins, labels=labels)` |
| **B** | `df_merged.groupby('Stage_Name')['FPRINCAM'].sum()` |
| **C** | `summary_agg.sort_values(by='Total_Principal', ascending=False)` |
| **D** | `df_merged['Stage_Name'] = df_merged['STAGE_CIF'].map(stage_map)` |
| **E** | `df_merged.pivot_table(index='DPD_Bucket', columns='FPRODTY', ...)` |
| **F** | `np.where(df_merged['FDPDUE01'] > 0, True, False)` |
| **G** | `(df_merged['FRPDATE'] - df_merged['FORDATE']).dt.days` |
| **H** | `df_merged.groupby('Stage_Name').agg(Total_Principal=('FPRINCAM', 'sum'), Avg_DPD=('FDPDUE01', 'mean'))` |

---

### เฉลย

| ข้อ | คำตอบ | คำอธิบาย |
|-----|-------|----------|
| 1 | **E** | pivot_table สร้างตารางไขว้ 2 มิติ |
| 2 | **D** | .map() แปลงค่าตาม dictionary |
| 3 | **F** | np.where() เหมือน IF ใน Excel |
| 4 | **B** | .groupby().sum() สรุปยอดรวม |
| 5 | **A** | pd.cut() จัดกลุ่มตัวเลขเป็นช่วง |
| 6 | **G** | การลบวันที่ + .dt.days แปลงเป็นจำนวนวัน |
| 7 | **C** | .sort_values() เรียงลำดับข้อมูล |
| 8 | **H** | .agg() กับ dictionary สรุปหลายแบบพร้อมกัน |

---

## What You Learned

- **Feature Engineering คือกุญแจสำคัญ:** การสร้างตัวแปรใหม่จากข้อมูลดิบทำให้วิเคราะห์ได้จริงและมีประโยชน์
- **เครื่องมือ Pandas ที่ใช้บ่อย:** `.map()` สำหรับแปลงค่า, `.apply()` สำหรับใช้ฟังก์ชันกับข้อมูล, `pd.cut()` สำหรับจัดกลุ่ม, และ `.dt` accessor สำหรับข้อมูลวันที่
- **การสรุปผลระดับมืออาชีพ:** ใช้ `groupby()` และ `pivot_table()` สร้างรายงานที่ผู้บริหารสามารถใช้ตัดสินใจได้ทันที

---